In [26]:
import os
from tree_sitter import Language, Parser

import tree_sitter_languages as tsl
language = tsl.get_language('javascript')
parser = tsl.get_parser('javascript')

def get_source_code(node, source_bytes):
    return source_bytes[node.start_byte:node.end_byte].decode('utf8')

def is_function_node(node):
    return node.type in ['function', 'function_declaration', 'arrow_function', 'method_definition']

def get_function_name(node, source_bytes):
    if node.type in ['function_declaration', 'function', 'method_definition']:
        name_node = node.child_by_field_name('name')
        if name_node is not None and name_node.type != 'MISSING':
            return get_source_code(name_node, source_bytes)
    return 'anonymous_function'

def traverse(node, source_bytes, function_calls, current_function=None):
    is_function = is_function_node(node)
    if is_function:
        function_name = get_function_name(node, source_bytes)
        function_source = get_source_code(node, source_bytes)
        if function_name not in function_calls:
            function_calls[function_name] = {
                'calls': [],
                'called_by': [],
                'source': function_source
            }
        current_function = function_name

    if node.type == 'call_expression':
        callee_node = node.child_by_field_name('callee')
        if callee_node is not None and callee_node.type != 'MISSING':
            called_function = get_source_code(callee_node, source_bytes)
            if current_function and called_function:
                function_calls[current_function]['calls'].append(called_function)
                if called_function not in function_calls:
                    function_calls[called_function] = {'calls': [], 'called_by': [], 'source': ''}
                function_calls[called_function]['called_by'].append(current_function)

    for child in node.children:
        traverse(child, source_bytes, function_calls, current_function if is_function else current_function)

def parse_file(file_path):
    with open(file_path, 'rb') as file:
        source_bytes = file.read()

    tree = parser.parse(source_bytes)
    function_calls = {}
    traverse(tree.root_node, source_bytes, function_calls)
    return function_calls

def parse_directory(directory_path):
    all_function_calls = {}
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.js'):
                file_path = os.path.join(root, file)
                all_function_calls[file] = parse_file(file_path)
    return all_function_calls

# Example usage
directory_path = './'
project_function_calls = parse_directory(directory_path)
print(project_function_calls)


{'example_file.js': {'anonymous_function': {'calls': [], 'called_by': [], 'source': '() => {\r\n    console.log("Arrow Function Called");\r\n}'}, 'constructor': {'calls': [], 'called_by': [], 'source': 'constructor() {\r\n        this.myMethod();\r\n    }'}, 'myMethod': {'calls': [], 'called_by': [], 'source': 'myMethod() {\r\n        externalFunction();\r\n    }'}, 'externalFunction': {'calls': [], 'called_by': [], 'source': 'function externalFunction() {\r\n    arrowFunc();\r\n}'}, 'shorthandMethod': {'calls': [], 'called_by': [], 'source': 'shorthandMethod() {\r\n        arrowFunc();\r\n    }'}}}


In [27]:
project_function_calls

{'example_file.js': {'anonymous_function': {'calls': [],
   'called_by': [],
   'source': '() => {\r\n    console.log("Arrow Function Called");\r\n}'},
  'constructor': {'calls': [],
   'called_by': [],
   'source': 'constructor() {\r\n        this.myMethod();\r\n    }'},
  'myMethod': {'calls': [],
   'called_by': [],
   'source': 'myMethod() {\r\n        externalFunction();\r\n    }'},
  'externalFunction': {'calls': [],
   'called_by': [],
   'source': 'function externalFunction() {\r\n    arrowFunc();\r\n}'},
  'shorthandMethod': {'calls': [],
   'called_by': [],
   'source': 'shorthandMethod() {\r\n        arrowFunc();\r\n    }'}}}